# Python Review Project

In [219]:
#SET WD
import os
os.chdir('/Users/andreavelazquez/Desktop/School stuff/QBIO490/qbio_490_AndreaV/analysis_data')

import cptac

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [220]:
#IMPORT CPTAC
cptac.download(dataset="Ccrcc")
ccrcc= cptac.Ccrcc()

In [221]:
#RETRIEVE DATA
#clinical
clinical_data = ccrcc.get_clinical()


#rna
rna_data = ccrcc.get_transcriptomics()


#proteins
protein_data = ccrcc.get_proteomics()
#for easier manipulation
protein_data.columns = protein_data.columns.get_level_values(0)

In [222]:
#CLEANUP
#na mask. Filter out patients with NaN for tumor stage
na_mask = clinical_data.loc[:, 'tumor_stage_pathological'].isna()
#apply mask to data frames
clinical_masked= clinical_data.loc[~na_mask, :]
rna_masked= rna_data.loc[~na_mask, :]
proteins_masked= protein_data.loc[~na_mask, :]


#change 0s to NAs before removing
rna_masked = rna_masked.replace(0, np.nan)
#get rid of genes with no data
gene_na_mask = rna_masked.isna().sum() == 0
#cleanup rna df
rna_masked = rna_masked.loc[:, gene_na_mask]


#same thing for proteins. get rid of the ones w no data
proteins_na_mask = proteins_masked.isna().sum() == 0
#cleanup df
proteins_masked = proteins_masked.loc[:, proteins_na_mask]


#stage mask to divide patients into Stage I cohort and stage III cohort
stg1_mask = clinical_masked.loc[:, 'tumor_stage_pathological'] == 'Stage I'
stg3_mask= clinical_masked.loc[:, 'tumor_stage_pathological'] == 'Stage III'

Y_data=clinical_masked.loc[: , 'tumor_stage_pathological']

#logscale
rna_masked= np.log2(rna_masked)


#TOP 5 MOST DIFFERENTIALLY EXPRESSED RNA 
#get top 5 differentially expressed genes for RNA stage1
s1_diff_rna = rna_masked.loc[stg1_mask, :].mean() - rna_masked.loc[stg3_mask, :].mean()
s1_t5_rna = np.abs(s1_diff_rna).sort_values(ascending=False)[:5].index

#TOP 5 MOST DIFFERENTIALLY EXPRESSED PROTEINS
#Stage 1
s1_diff_proteins= proteins_masked.loc[stg1_mask, :].mean()- proteins_masked.loc[stg3_mask, :].mean()
s1_t5_proteins= np.abs(s1_diff_proteins).sort_values(ascending= False)[:5].index




In [223]:
#MAKE NEW DF
s1s3_rna= s1_t5_rna[:]
s1s3_rna

s1s3_proteins= s1_t5_proteins[:]

#rows =patients and the columns =expression values of selected genes
s1s3_newdf = rna_masked[s1s3_rna]

#add protein data
for i in range(5):
    s1s3_newdf.loc[: , s1s3_proteins[i]]= proteins_masked.loc[: , s1s3_proteins[i]]
    
s1s3_newdf

/var/folders/5x/ysrbhc4j6hldh0hl5gpqh_3m0000gn/T/ipykernel_37361/4117901636.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1s3_newdf.loc[: , s1s3_proteins[i]]= proteins_masked.loc[: , s1s3_proteins[i]]
/var/folders/5x/ysrbhc4j6hldh0hl5gpqh_3m0000gn/T/ipykernel_37361/4117901636.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  s1s3_newdf.loc[: , s1s3_proteins[i]]= proteins_masked.loc[: , s1s3_proteins[i]]
/var/folders/5x/ysrbhc4j6hldh0hl5gpqh_3m0000gn/T/ipykernel_37361/4117901636.py:12: SettingWith

Name,AJAP1,DPEP1,GALNT5,IL20RB,RYR2,FTL,HBZ,HBA2,CMA1,HBB
Patient_ID,,,,,,,,,,
C3L-00004,-2.519207,-1.239811,-5.281123,2.685910,-0.721420,-1.724339,-0.370098,-0.786307,-0.918273,-0.762287
C3L-00010,-0.069282,4.515644,-1.685391,0.394540,-2.134346,-0.363228,0.240576,-0.081263,0.747860,-0.034944
C3L-00011,0.411676,-1.955016,-2.430473,5.329663,3.102518,-0.977364,-0.087641,-0.418663,-0.255054,-0.371128
C3L-00026,0.177869,4.635430,2.729468,1.139609,-1.565029,1.301930,0.981930,0.780756,0.049086,0.855097
C3L-00079,-1.161677,0.376214,-3.413559,0.441469,-0.421905,-1.496648,-0.441854,-1.014088,-0.620829,-0.939448
...,...,...,...,...,...,...,...,...,...,...
C3N-01646,-1.251963,-0.946572,-0.635367,7.399686,-0.645290,-1.817120,-0.724017,-1.180566,-0.000699,-1.127855
C3N-01648,-2.212814,2.734819,1.865296,0.789974,-1.221495,0.459593,-0.396638,-0.672383,-0.611409,-0.605164
C3N-01649,-0.097150,3.354027,-1.442680,-0.220637,2.057367,-1.066512,-0.109630,-0.584516,0.322161,-0.527777


In [224]:
#ENCODE AND SCALE
scaler = StandardScaler()
s1s3_newdf = scaler.fit_transform(s1s3_newdf)

encoder= LabelEncoder()
Y_data= encoder.fit_transform(Y_data)

In [231]:
#SPLITS
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(s1s3_newdf, Y_data, train_size=0.7)

#ACCURACY: KNeighborsClassifier, DecisionTreeClassifier, and MLPClassifier, GaussianNB
from sklearn.neighbors import KNeighborsClassifier # default number of neighbors looked at is 5
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

#KNeighborsClassifier
def avg_acc_classifier(funct):
    
    #list to save results
    results=[]
    
    for i in range(10):
        #Split df into training and testing. Diff split each time
        X_train, X_test, y_train, y_test = train_test_split(s1s3_newdf, Y_data, train_size=0.7)

        #use classifier 
        classifier = funct
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        accuracy = sum(y_pred == y_test) / len(y_test)
        results.append(accuracy)
    
    avg= sum(results)/len(results)

    print("Average accuracy for the given classifier: ", avg)
    
    return 

KNN= avg_acc_classifier(KNeighborsClassifier())
DT= avg_acc_classifier(DecisionTreeClassifier())
MLP= avg_acc_classifier(MLPClassifier())
GNB= avg_acc_classifier(GaussianNB())


Average accuracy for the given classifier:  0.49696969696969706
Average accuracy for the given classifier:  0.3939393939393939


/Users/andreavelazquez/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/andreavelazquez/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/andreavelazquez/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/andreavelazquez/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.wa

Average accuracy for the given classifier:  0.5272727272727273
Average accuracy for the given classifier:  0.5666666666666667


/Users/andreavelazquez/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/andreavelazquez/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
